
# SomPyte — FAW CV5 (ResNet18/34) — **Auto‑Path**
This version **auto‑detects** `Train.csv`, `Test.csv` and the `Images/` folder anywhere inside your cloned repo.
It then trains 5‑fold CV for **ResNet18** and **ResNet34** and saves:
- `submission_cv5_resnet18.csv`
- `submission_cv5_resnet34.csv`
- `submission_blend_r18_r34.csv`

> Works with: `https://github.com/Gaabshiine/pycon-2025-hackthon.git`


In [ ]:

# %% [setup] (install minimal deps if missing)
import sys, subprocess

def pip_install(pkg):
    try:
        __import__(pkg.split('==')[0].split('[')[0])
    except Exception:
        print(f"Installing: {pkg}")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pkg])

for p in ["tqdm", "scikit-learn", "torchvision", "pandas", "numpy", "Pillow"]:
    pip_install(p)
print("✅ deps ready")


In [ ]:

# %% [imports]
import os, math, random, time, gc
from pathlib import Path

import numpy as np
import pandas as pd
from PIL import Image

from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.transforms as T

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", DEVICE)


In [ ]:

# %% [paths] Auto-detect Train.csv/Test.csv/Images anywhere in repo
def find_data_root(start: Path):
    # search for Train.csv and Test.csv (case-insensitive) and an Images folder
    lower_ok = False
    for p in [start, *start.rglob("*")]:
        train_u = p / "Train.csv"
        test_u  = p / "Test.csv"
        images_u = p / "Images"
        train_l = p / "train.csv"
        test_l  = p / "test.csv"
        images_l = p / "images"
        if train_u.exists() and test_u.exists() and (images_u.exists() or images_l.exists()):
            return p, train_u, test_u, images_u if images_u.exists() else images_l
        if train_l.exists() and test_l.exists() and (images_u.exists() or images_l.exists()):
            lower_ok = True
            return p, train_l, test_l, images_u if images_u.exists() else images_l
    raise FileNotFoundError("Could not find Train.csv/Test.csv/Images folder. Please verify repo structure.")

REPO_ROOT = Path.cwd()
DATA_DIR, TRAIN_CSV, TEST_CSV, IMAGES_DIR = find_data_root(REPO_ROOT)
print("DATA_DIR =", DATA_DIR)
print("TRAIN   =", TRAIN_CSV)
print("TEST    =", TEST_CSV)
print("IMAGES  =", IMAGES_DIR)


In [ ]:

# %% [config]
SEED = 42
IMG_SIZE = 256
BATCH_SIZE = 48
EPOCHS = 12
N_FOLDS = 5
TTA = True  # horizontal flip only

def set_seed(seed=SEED):
    import random, numpy as np, torch
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(SEED)


In [ ]:

# %% [load dfs]
df_train = pd.read_csv(TRAIN_CSV)
df_test  = pd.read_csv(TEST_CSV)
print(df_train.shape, df_test.shape)
df_train.head()


In [ ]:

# %% [dataset]
class FAWDataset(Dataset):
    def __init__(self, df, root, train=True, transforms=None):
        self.df = df.reset_index(drop=True)
        self.root = Path(root)
        self.train = train
        self.transforms = transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = self.root / row["Image_id"]
        img = Image.open(img_path).convert("RGB")
        if self.transforms:
            img = self.transforms(img)
        if self.train:
            y = torch.tensor(row["Label"], dtype=torch.float32)
            return img, y
        else:
            return img, row["Image_id"]


In [ ]:

# %% [transforms]
train_tfms = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE)),
    T.RandomHorizontalFlip(p=0.5),
    T.RandomRotation(12),
    T.RandomPerspective(distortion_scale=0.15, p=0.25),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
valid_tfms = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [ ]:

# %% [model + ema]
def build_model(backbone="resnet18"):
    if backbone == "resnet18":
        m = torchvision.models.resnet18(weights=torchvision.models.ResNet18_Weights.IMAGENET1K_V1)
    elif backbone == "resnet34":
        m = torchvision.models.resnet34(weights=torchvision.models.ResNet34_Weights.IMAGENET1K_V1)
    else:
        raise ValueError("backbone must be resnet18 or resnet34")
    in_feats = m.fc.in_features
    m.fc = nn.Linear(in_feats, 1)
    return m

class EMA:
    def __init__(self, model, decay=0.999):
        self.decay = decay
        self.shadow = {n: p.data.clone() for n,p in model.named_parameters() if p.requires_grad}
    def update(self, model):
        for n,p in model.named_parameters():
            if p.requires_grad:
                self.shadow[n] = (1-self.decay)*p.data + self.decay*self.shadow[n]
    def apply_shadow(self, model):
        self.bak = {}
        for n,p in model.named_parameters():
            if p.requires_grad:
                self.bak[n] = p.data.clone()
                p.data = self.shadow[n].clone()
    def restore(self, model):
        for n,p in model.named_parameters():
            if p.requires_grad and n in self.bak:
                p.data = self.bak[n]
        self.bak = {}


In [ ]:

# %% [train/valid]
def train_one_epoch(model, loader, optimizer, scaler):
    model.train()
    losses = []
    for imgs, y in loader:
        imgs = imgs.to(DEVICE, non_blocking=True)
        y = y.to(DEVICE, non_blocking=True).unsqueeze(1)
        optimizer.zero_grad(set_to_none=True)
        with torch.amp.autocast(device_type="cuda", enabled=(DEVICE=="cuda")):
            logits = model(imgs)
            loss = F.binary_cross_entropy_with_logits(logits, y)
        scaler.scale(loss).backward()
        scaler.step(optimizer); scaler.update()
        losses.append(loss.detach().item())
    return float(np.mean(losses))

@torch.no_grad()
def valid_one_epoch(model, loader):
    model.eval()
    probs, targs = [], []
    for imgs, y in loader:
        imgs = imgs.to(DEVICE, non_blocking=True)
        p = torch.sigmoid(model(imgs)).squeeze(1).detach().cpu().numpy()
        probs.append(p); targs.append(y.numpy())
    probs = np.concatenate(probs); targs = np.concatenate(targs)
    auc = roc_auc_score(targs, probs)
    return auc


In [ ]:

# %% [cv5]
def run_fold(df, fold, backbone="resnet18"):
    train_idx = df.index[df.kfold != fold]
    valid_idx = df.index[df.kfold == fold]
    dtr = df.loc[train_idx].reset_index(drop=True)
    dvl = df.loc[valid_idx].reset_index(drop=True)

    tr_ds = FAWDataset(dtr, IMAGES_DIR, True, train_tfms)
    vl_ds = FAWDataset(dvl, IMAGES_DIR, True, valid_tfms)
    te_ds = FAWDataset(df_test, IMAGES_DIR, False, valid_tfms)

    tr = DataLoader(tr_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, pin_memory=True)
    vl = DataLoader(vl_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)
    te = DataLoader(te_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)

    model = build_model(backbone).to(DEVICE)
    opt = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)
    sch = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=EPOCHS)
    scaler = torch.amp.GradScaler(enabled=(DEVICE=="cuda"))
    ema = EMA(model, decay=0.999)

    best_auc, best_path, patience = -1.0, f"best_{backbone}_fold{fold}.pt", 0
    for ep in range(1, EPOCHS+1):
        tr_loss = train_one_epoch(model, tr, opt, scaler)
        ema.update(model)
        ema.apply_shadow(model); val_auc = valid_one_epoch(model, vl); ema.restore(model)
        sch.step()
        print(f"[{backbone}][fold {fold}][ep {ep}] loss={tr_loss:.4f} AUC={val_auc:.6f}")
        if val_auc > best_auc:
            best_auc, patience = val_auc, 0
            torch.save(model.state_dict(), best_path)
        else:
            patience += 1
            if patience >= 3: 
                print("Early stop."); break

    model.load_state_dict(torch.load(best_path, map_location=DEVICE)); model.eval()
    # OOF
    ema.apply_shadow(model)
    v_probs = []
    for imgs, y in vl:
        imgs = imgs.to(DEVICE, non_blocking=True)
        v_probs.append(torch.sigmoid(model(imgs)).squeeze(1).cpu().numpy())
    ema.restore(model)
    v_probs = np.concatenate(v_probs)
    oof = pd.DataFrame({"idx": valid_idx, "oof": v_probs}).set_index("idx")

    # TEST (+flip TTA)
    te_probs = []
    with torch.no_grad():
        for imgs, ids in te:
            imgs = imgs.to(DEVICE, non_blocking=True)
            p = torch.sigmoid(model(imgs)).squeeze(1)
            if True:
                p = 0.5*(p + torch.sigmoid(model(torch.flip(imgs, dims=[3]))).squeeze(1))
            te_probs.append(p.cpu().numpy())
    te_probs = np.concatenate(te_probs)
    return best_auc, oof, te_probs


In [ ]:

# %% [split]
df = df_train.copy()
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
df["kfold"] = -1
for f, (_, v) in enumerate(skf.split(df, df["Label"])):
    df.loc[v, "kfold"] = f
df["kfold"].value_counts().sort_index()


In [ ]:

# %% [run resnet18]
bk = "resnet18"
all_oof = np.zeros(len(df)); test_pred = np.zeros(len(df_test))
fold_aucs = []
for f in range(5):
    auc, oof, te = run_fold(df, f, backbone=bk)
    fold_aucs.append(auc); all_oof[oof.index.values] = oof["oof"].values; test_pred += te/5
print(f"{bk} fold AUCs:", fold_aucs, "mean:", np.mean(fold_aucs))
print("OOF AUC:", roc_auc_score(df["Label"].values, all_oof))
pd.DataFrame({"Image_id": df_test["Image_id"], "Label": test_pred}).to_csv("submission_cv5_resnet18.csv", index=False)
print("Saved submission_cv5_resnet18.csv")


In [ ]:

# %% [run resnet34]
bk = "resnet34"
all_oof = np.zeros(len(df)); test_pred = np.zeros(len(df_test))
fold_aucs = []
for f in range(5):
    auc, oof, te = run_fold(df, f, backbone=bk)
    fold_aucs.append(auc); all_oof[oof.index.values] = oof["oof"].values; test_pred += te/5
print(f"{bk} fold AUCs:", fold_aucs, "mean:", np.mean(fold_aucs))
print("OOF AUC:", roc_auc_score(df["Label"].values, all_oof))
pd.DataFrame({"Image_id": df_test["Image_id"], "Label": test_pred}).to_csv("submission_cv5_resnet34.csv", index=False)
print("Saved submission_cv5_resnet34.csv")


In [ ]:

# %% [blend]
import pandas as pd
a = pd.read_csv("submission_cv5_resnet18.csv")
b = pd.read_csv("submission_cv5_resnet34.csv")
m = a.merge(b, on="Image_id", suffixes=("_r18","_r34"))
m["Label"] = 0.5*m["Label_r18"] + 0.5*m["Label_r34"]
m[["Image_id","Label"]].to_csv("submission_blend_r18_r34.csv", index=False)
print("Saved submission_blend_r18_r34.csv")
